In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

In [2]:
local_llm = LLM(
    model="ollama/llama3.1",
    base_url="http://localhost:11434"
)

cloud_llm = LLM(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)


In [3]:
# Instantiate the tools

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [4]:
# Agents

data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    llm=cloud_llm,
    tools = [scrape_tool, search_tool]
)

trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    llm=cloud_llm,
    tools = [scrape_tool, search_tool]
)

execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    llm=cloud_llm,
    tools = [scrape_tool, search_tool]
)

risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    llm=cloud_llm,
    tools = [scrape_tool, search_tool]
)

In [5]:
# Tasks

# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


In [6]:
# Crew

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=cloud_llm,
    process=Process.hierarchical,
    verbose=True
)


In [7]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'WIT',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 09fc6b38-f195-42cd-856c-8a08954bf103                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Continuously monitor and analyze market data for the selected stock (WIT). Use statistical modeling and  │
│  machine learning to identify trends and predict market movements.                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Task: Perform a comprehensive market data analysis for the stock 'WIT'. This analysis should include           │
│  identifying current trends, predicting market movements using statistical modeling and machine learning        │
│  techniques, and generating insights and alerts about significant market opportunities or threats. The goal is  │
│  to provide a snapshot of the current market situation for WIT based on your analysis.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  ### Risk Assessment and Mitigation Strategies for Wipro Limited (WIT) Day Trading                              │
│                                                                                                                 │
│  **I. Executive Summary of WIT's Risk Profile**                                                                 │
│                                                                                                                 │
│  Wipro Limited (WIT) presents a complex risk profile for day trading, characterized by conflicting signals.     │
│  While strategic initiatives in AI and digital transformation, improved net income, and several short-term      │
│  technical "buy signals" (Moving Averages, MACD, pivot bottom) suggest underlying strength and potential for    │
│  intraday bounces, these are significantly counterbalanced by a prevailing short-term falling trend and a       │
│  concerning bearish 3-month forecast from an AI analyst (-3.70% decline). The stock is trading near its         │
│  52-week low, and revenue growth concerns persist.                                                              │
│                                                                                                                 │
│  Given the 'Medium' risk tolerance and 'Day Trading' preferences, the primary challenge is to capitalize on     │
│  short-term technical opportunities while rigidly mitigating exposure to the broader bearish sentiment and the  │
│  predicted medium-term decline. The low daily volatility (average 1.49%) implies that profit targets will be    │
│  small, making precise execution and stringent risk management paramount to offset transaction costs and avoid  │
│  significant losses.                                                                                            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **II. General Risks of Day Trading for WIT (Medium Risk Tolerance)**                                           │
│                                                                                                                 │
│  1.  **Conflicting Market Signals**: The presence of both short-term bullish technicals and a strong bearish    │
│  fundamental outlook (AI analyst forecast, falling trend) creates an environment of high uncertainty. This      │
│  increases the risk of whipsaws and false signals, which can lead to multiple stop-outs.                        │
│      *   **Mitigation**: Prioritize clear confirmation signals for entries and exits. Avoid trading during      │
│  periods of extreme indecision or when conflicting signals are strong without clear dominance.                  │
│  2.  **Low Volatility and Tight Ranges**: With an average daily volatility of 1.49% and a predicted intraday    │
│  range of $2.71-$2.75, profit margins per trade are inherently small. This increases the relative impact of     │
│  transaction costs (commissions, spread...                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
├── 📋 Task: 63bade45-2eea-4d30-b2a6-911321bfc1e3
│   Assigned to: Crew Manager
│   Status: ✅ Completed
│   ├── 🔧 Using Delegate work to coworker (1)
│   ├── 🔧 Used Search the internet with Serper (1)
│   ├── 🔧 Used Read website content (1)
│   ├── 🔧 Used Read website content (2)
│   ├── 🔧 Used Read website content (3)
│   ├── 🔧 Used Search the internet with Serper (2)
│   ├── 🔧 Used Read website content (4)
│   └── 🔧 Using Delegate work to coworker (2)
├── 📋 Task: 7280790a-710e-44fa-8078-51b220ccab45
│   Assigned to: Crew Manager
│   Status: ✅ Completed
├── 📋 Task: f4366d32-2814-47bb-b494-cd2d5b88ee9e
│   Assigned to: Crew Manager
│   Status: ✅ Completed
└── 📋 Task: c6b53806-185b-4caf-b6b3-f71faeb36ab2
    Status: Executing Task...
    └── 🔧 Using Delegate work to coworker (3)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c6b53806-185b-4caf-b6b3-f71faeb36ab2                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 09fc6b38-f195-42cd-856c-8a08954bf103                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Risk Assessment and Mitigation Strategies for Wipro Limited (WIT) Day Trading                │
│                                                                                                                 │
│  **I. Executive Summary of WIT's Risk Profile**                                                                 │
│                                                                                                                 │
│  Wipro Limited (WIT) presents a complex risk profile for day trading, characterized by conflicting signals.     │
│  While strategic initiatives in AI and digital transformation, improved net income, and several short-term      │
│  technical "buy signals" (Moving Averages, MACD, pivot bottom) suggest underlying strength and potential for    │
│  intraday bounces, these are significantly counterbalanced by a prevailing short-term falling trend and a       │
│  concerning bearish 3-month forecast from an AI analyst (-3.70% decline). The stock is trading near its         │
│  52-week low, and revenue growth concerns persist.                                                              │
│                                                                                                                 │
│  Given the 'Medium' risk tolerance and 'Day Trading' preferences, the primary challenge is to capitalize on     │
│  short-term technical opportunities while rigidly mitigating exposure to the broader bearish sentiment and the  │
│  predicted medium-term decline. The low daily volatility (average 1.49%) implies that profit targets will be    │
│  small, making precise execution and stringent risk management paramount to offset transaction costs and avoid  │
│  significant losses.                                                                                            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **II. General Risks of Day Trading for WIT (Medium Risk Tolerance)**                                           │
│                                                                                                                 │
│  1.  **Conflicting Market Signals**: The presence of both short-term bullish technicals and a strong bearish    │
│  fundamental outlook (AI analyst forecast, falling trend) creates an environment of high uncertainty. This      │
│  increases the risk of whipsaws and false signals, which can lead to multiple stop-outs.                        │
│      *   **Mitigation**: Prioritize clear confirmation signals for entries and exits. Avoid trading during      │
│  periods of extreme indecision or when conflicting signals are strong without clear dominance.                  │
│  2.  **Low Volatility and Tight Ranges**: With an average daily volatility of 1.49% and a predicted intraday    │
│  range of $2.71-$2.75, profit margins per trade are inherently small. This increases the relative impact of     │
│  transaction costs (commissions, spreads) on profitabi

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯